In [2]:
import nagisa
import requests

with open("text.txt", "r") as file:
    text = str.join("", file.readlines())

In [4]:
print(text)

「さっぽろ雪まつり」きょう開幕 新型肺炎や雪不足の影響も
2020年2月4日 5時28分

雪や氷の像で冬の札幌を彩る「さっぽろ雪まつり」が４日、市の中心部のメイン会場で開幕します。ことしは新型コロナウイルスの感染拡大や雪不足の影響を受ける中での開催となります。

ことしで71回目を迎える「さっぽろ雪まつり」は、一足早く始まっている札幌市東区の「つどーむ会場」に続き、４日、メインとなる札幌市中心部の大通会場と、繁華街のすすきの会場でも開幕し、およそ200基の雪像や氷像が展示されます。

大通会場では、ポーランドの宮殿を実際の半分の大きさで再現した雪像や、北海道のサラブレッドをテーマにした高さ12ｍの雪像など、５基の大雪像が登場します。

すすきの会場にはおよそ60基の氷像が展示され、つどーむ会場では雪で作った滑り台も楽しめます。

さっぽろ雪まつりは去年は270万人余りが訪れた札幌の冬の一大イベントです。

３つの会場とも今月11日まで行われます。

「さっぽろ雪まつり」 ホテルキャンセル相次ぐ
札幌市中央区のホテル「ニューオータニイン札幌」では、「さっぽろ雪まつり」の時期は例年、客室の稼働率が95％程度と満室に近い状態でした。

しかし、ことしは新型コロナウイルウスの感染の拡大を防ぐため中国政府が団体旅行を禁止している影響で、先月末からキャンセルが相次ぎました。

これまでにおよそ1500件の予約がキャンセルされ、稼働率は80％程度にとどまっているということです。

客室の予約状況を示す画面でも、空室が10室以上ある○印の部屋が多く見られます。

値段もこの時期の通常より30％程度安くしているということです。

松島啓二・営業企画室長は「中国の春節から雪まつりというこの時期は、通常だとかきいれどきなので、今回の影響はかなり大きいです。ほかのホテルの動向を見て価格設定の見直しをはかっていきたい」と話していました。

札幌市の試算によりますと、市内のホテルや旅館などのキャンセルは、３月末までに13万3000人にのぼっています。

これは調査対象のホテルの客室すべての丸２日分にあたり、売り上げの減少はおよそ64億円に達する見込みだということです。
新型コロナウイルス 対策あれこれ
雪まつりの実行委員会は、会場が国内外の多くの人で混雑するため、新型コロナウイルスの対策も行

In [16]:
tags = nagisa.tagging(text)
words = tags.words
pos = tags.postags

In [63]:
short = "今日はいい天気ですね。"
url = "https://jisho.org/api/v1/search/words?keyword={keyword}"

In [72]:
# def word_to_entry(word):
#     japanese = word["japanese"][0]
#     return f'{japanese["word"]} 「{japanese["reading"]}」: {str.join("; ", [str.join(", ", sense["english_definitions"]) for sense in word["senses"]])}\n'

def word_to_entry(word):
    japanese = word["japanese"][0]
    senses = word["senses"]
    return f'{japanese["word"]}\t{japanese["reading"]}\t{str.join("; ", [str.join(", ", sense["english_definitions"]) for sense in senses])}'

In [ ]:
tags = nagisa.tagging(text)
for word, tag in zip(tags.words, tags.postags):
    if tag in ["助詞", "補助記号"]:
        continue
    result = requests.get(url.format(keyword=word)).json()
    try:
        print(word_to_entry(result["data"][0]))
    except:
        print(f"Error! {word}")

札幌	さっぽろ	Sapporo (city)
雪祭り	ゆきまつり	snow festival
今日	きょう	today, this day; these days, recently, nowadays


In [80]:
[(word, tag) for word, tag in zip(tags.words, tags.postags)]


[('今日', '名詞'),
 ('は', '助詞'),
 ('いい', '形容詞'),
 ('天気', '名詞'),
 ('です', '助動詞'),
 ('ね', '助詞'),
 ('。', '補助記号')]